In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

rng = np.random.default_rng(42)
n = 1000

rooms = rng.integers(1, 4, size=n)

area = (
    rng.normal(30, 5, size=n) * (rooms == 1) +
    rng.normal(50, 7, size=n) * (rooms == 2) +
    rng.normal(75, 10, size=n) * (rooms == 3)
)

floor = rng.integers(1, 26, size=n)

base_price_per_m2 = (
    120_000 * (rooms == 1) +
    140_000 * (rooms == 2) +
    160_000 * (rooms == 3)
)
noise = rng.normal(0, 300_000, size=n)
price = area * base_price_per_m2 + noise

df = pd.DataFrame({
    'price': price.round(0),
    'area': area.round(1),
    'rooms': rooms,
    'floor': floor,
})
df.head()

,price,area,rooms,floor
0,4149512.0,37.2,1,23
1,11878793.0,77.2,3,5
2,7389165.0,52.3,2,8
3,7832436.0,55.4,2,25
4,6716941.0,44.3,2,9


In [ ]:
q1, q2 = df['price'].quantile([0.33, 0.66])

def price_to_class(p):
    if p <= q1:
        return 0
    if p <= q2:
        return 1
    return 2

df['price_class'] = df['price'].apply(price_to_class)

X = df[['area', 'rooms', 'floor']].values
y = df['price_class'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)
len(X_train), len(X_test)

(600, 400)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Точность: {acc:.2f}')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2057 - loss: 1.3345
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3180 - loss: 1.0916
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5165 - loss: 0.9136
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5785 - loss: 0.7794
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6774 - loss: 0.6736
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8113 - loss: 0.6001
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9057 - loss: 0.4771
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9318 - loss: 0.3728
Epoch 9/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9579 - loss: 0.2859
Epoch 10/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9654 - loss: 0.2512
Epoch 11/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9727 - loss: 0.1998
Epoch 12/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9756 - lo